In [11]:
import numpy as np
import pandas as pd
import string
import re

import seaborn as sns
pd.set_option('display.max_columns', None)

In [12]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
data_path = "data/"

In [14]:
player_stat = pd.read_excel(data_path + "players stat.xlsx")
player_cv = pd.read_excel(data_path + "players cv.xlsx")
player_salary = pd.read_excel(data_path + "players salary.xlsx")

### 2014-2015 season for player_stat

In [15]:
player_stat_14_15 = player_stat[player_stat['Season'] == '2014-15']
player_stat_14_15.shape

(492, 29)

In [16]:
player_stat_14_15['Player'].value_counts()

Quincy Acy          1
Shabazz Napier      1
Toure' Murry        1
Shabazz Muhammad    1
Timofey Mozgov      1
                   ..
Joel Freeland       1
Jimmer Fredette     1
Tim Frazier         1
Jamaal Franklin     1
Tyler Zeller        1
Name: Player, Length: 492, dtype: int64

<div class="alert alert-success">

- Comment: no duplicated names for player_stat_14_15

### Get 2014-2015 data for player_cv

In [17]:
player_cv_14_15 = player_cv[(player_cv['From']<=2015) & (player_cv['To']>=2014)]

In [18]:
player_cv_14_15.shape

(582, 10)

### Merge these two

In [19]:
merge_stat_cv = pd.merge(player_stat_14_15, player_cv_14_15, left_on='Player', right_on='Player', how='inner')

In [20]:
merge_stat_cv.shape[0] == player_stat_14_15.shape[0]

True

<div class="alert alert-success">

- Comment: $\{$player $P$: $P \in$ player_stat_14_15$\}$  $\subseteq$ \{ player $P$: $P \in$ player_cv_14_15 \}

### Get 2014-2015 data for player_salary

In [21]:
player_salary_14_15 = player_salary[player_salary['SEASON'] == '2014-2015']

In [22]:
player_salary_14_15.shape

(432, 5)

In [23]:
player_salary_14_15.head(3)

,SEASON,RK,NAME,TEAM,SALARY
158,2014-2015,41,"Al Horford, C",Atlanta Hawks,12000000
159,2014-2015,59,"Paul Millsap, PF",Atlanta Hawks,9500000
160,2014-2015,74,"Jeff Teague, PG",Atlanta Hawks,8000000


#### preprocess the name

In [24]:
# all names contain ","
player_salary_14_15['NAME'].str.contains(",").sum() == player_salary_14_15.shape[0]

True

In [25]:
# all names contain one and only one ","
for l in player_salary_14_15['NAME'].str.split(",").to_list():
    if len(l)!=2:
        print(l)

<div class="alert alert-success">

- Comment: We noticed that in player_salary_14_15, these four players' names are not in good format: Jeff Taylor, Louis Williams, Luc Richard Mbah A Moute, Patty Mills. For example, in the table merge_stat_cv, Jeff Taylor is called Jeffery Taylor.

In [26]:
false_names = ['jeff taylor', 'louis williams', 'luc richard mbah a moute', 'patty mills']
true_names = ['jeffery taylor', 'lou williams', 'luc mbah a moute', 'patrick mills']

In [27]:
def preprocess_name(name):
    if ',' in name:
        ind = name.find(',')
        name = name[:ind]
    ### We remove the Jr.
    if "Jr." in name:
        name = name.replace(" Jr.", "")
    if "Jr" in name:
        name = name.replace(" Jr.", "")
    if "III" in name:
        name = name.replace(" III", "")
    while '.' in name:
        ind = name.find('.')
        name = name.replace('.','')
        
    name = name.lower()
    name = name.strip()
    
    if name in false_names:
        name = true_names[false_names.index(name)]
    
    return name

## Final dataframe to work on for 2014-2015

In [28]:
player_salary_14_15['Player'] = player_salary_14_15['NAME'].apply(lambda s: preprocess_name(s))

In [29]:
merge_stat_cv['Player'] = merge_stat_cv['Player'].apply(lambda s: preprocess_name(s))

In [33]:
merge_14_15 = pd.merge(merge_stat_cv, player_salary_14_15, left_on='Player', right_on='Player', how='inner')
merge_14_15.shape

(406, 43)

In [34]:
merge_14_15.head()

,Player,Season,Age,Tm,Lg,G,GS,MP,PER,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Place_of_Birth,Race,From,To,Pos,Ht,Wt,Birth Date,College,SEASON,RK,NAME,TEAM,SALARY
0,jordan adams,2014-15,20.0,MEM,NBA,30,0.0,248.0,12.8,0.291,0.267,4.2,8.7,6.4,10.1,3.4,2.3,12.7,20.4,96.0,100.0,0.0,0.4,0.4,0.073,-1.8,1.2,-0.6,0.1,Georgia,Black,2015,2016,G,6-5,209,"July 8, 1994","University of California, Los Angeles",2014-2015,299,"Jordan Adams, SG",Memphis Grizzlies,1344120
1,steven adams,2014-15,21.0,OKC,NBA,70,67.0,1771.0,14.1,0.005,0.514,12.2,19.3,15.8,5.5,1.1,3.8,16.8,14.3,108.0,104.0,1.9,2.2,4.1,0.111,-1.4,1.8,0.4,1.1,New zealand,White,2014,2016,C,7-0,255,"July 20, 1993",University of Pittsburgh,2014-2015,242,"Steven Adams, C",Oklahoma City Thunder,2184960
2,jeff adrien,2014-15,28.0,MIN,NBA,17,0.0,215.0,14.2,0.000,0.864,11.9,29.6,20.5,10.5,0.9,3.3,12.9,14.3,109.0,108.0,0.2,0.2,0.4,0.087,-2.7,0.5,-2.2,0.0,Massachusetts,Black,2011,2015,F,6-7,245,"February 10, 1986",University of Connecticut,2014-2015,377,"Jeff Adrien, SF",Minnesota Timberwolves,742962
3,arron afflalo,2014-15,29.0,TOT,NBA,78,72.0,2502.0,10.7,0.377,0.224,1.1,9.7,5.3,8.2,0.8,0.2,10.7,19.0,103.0,111.0,1.6,1.0,2.6,0.050,-0.5,-1.3,-1.8,0.1,California,Black,2008,2016,G,6-5,210,"October 15, 1985","University of California, Los Angeles",2014-2015,79,"Arron Afflalo, SG",Portland Trail Blazers,7500000
4,alexis ajinca,2014-15,26.0,NOP,NBA,68,8.0,957.0,19.9,0.000,0.301,12.4,25.0,18.7,8.2,1.2,4.0,15.6,21.1,113.0,103.0,1.9,1.2,3.2,0.159,-0.5,0.6,0.2,0.5,France,White,2009,2016,C,7-2,248,"May 6, 1988",NaN,2014-2015,339,"Alexis Ajinca, C",New Orleans Pelicans,981084


In [36]:
np.sort(merge_14_15.TEAM.unique())

array(['Atlanta Hawks', 'Boston Celtics', 'Brooklyn Nets',
       'Charlotte Hornets', 'Chicago Bulls', 'Cleveland Cavaliers',
       'Dallas Mavericks', 'Denver Nuggets', 'Detroit Pistons',
       'Golden State Warriors', 'Houston Rockets', 'Indiana Pacers',
       'Los Angeles Clippers', 'Los Angeles Lakers', 'Memphis Grizzlies',
       'Miami Heat', 'Milwaukee Bucks', 'Minnesota Timberwolves',
       'New Orleans Pelicans', 'New York Knicks', 'Oklahoma City Thunder',
       'Orlando Magic', 'Philadelphia 76ers', 'Phoenix Suns',
       'Portland Trail Blazers', 'Sacramento Kings', 'San Antonio Spurs',
       'Toronto Raptors', 'Utah Jazz', 'Washington Wizards'], dtype=object)

In [38]:
merge_14_15.to_csv()

<function pandas.core.generic.NDFrame.to_csv(self, path_or_buf: 'FilePathOrBuffer[AnyStr] | None' = None, sep: 'str' = ',', na_rep: 'str' = '', float_format: 'str | None' = None, columns: 'Sequence[Hashable] | None' = None, header: 'bool_t | list[str]' = True, index: 'bool_t' = True, index_label: 'IndexLabel | None' = None, mode: 'str' = 'w', encoding: 'str | None' = None, compression: 'CompressionOptions' = 'infer', quoting: 'int | None' = None, quotechar: 'str' = '"', line_terminator: 'str | None' = None, chunksize: 'int | None' = None, date_format: 'str | None' = None, doublequote: 'bool_t' = True, escapechar: 'str | None' = None, decimal: 'str' = '.', errors: 'str' = 'strict', storage_options: 'StorageOptions' = None) -> 'str | None'>